#Swarm Optimization

This is a program from optimizing functions using swarm optimization. Swarm optimization is basically that you randomly initiate several "solutions" to your problem to create the swarm. Each "solution" in the swarm has a velocity that will determine where the “solution” will "land" the next iteration. The "solution" remembers its best location (in terms of the objective function) but can also so its neighbors best position. Its velocity is updated each time step based on this knowledge.

In [79]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
import imageio

The objective function will be the Townsend function (modified) https://en.wikipedia.org/wiki/Test_functions_for_optimization

In [80]:
class Objective:
  def __init__(self,dim,bounds,name,anal_sol,anal_obj,barrier_fac):
    self.dim = dim
    self.bounds = bounds
    self.name = name
    self.anal_sol = anal_sol
    self.anal_obj = anal_obj
    if barrier_fac == None:
      self.barrier_fac = 0
    else:
      self.barrier_fac = barrier_fac
  
  def func(self, x):
    t = np.arctan2(x[0],x[1])
    constr = -(2*np.cos(t)-1/2*np.cos(2*t)-1/4*np.cos(3*t)-1/8*np.cos(4*t))**2 - (2*np.sin(t))**2 + x[0]**2 + x[1]**2
    barrier = self.barrier_fac*(np.maximum(0,(constr)))**2
    return -(np.cos((x[0]-0.1)*x[1]))**2-x[0]*np.sin(3*x[0]+x[1]) + barrier


In [81]:
Obj = Objective(2,None,'Townsend function (modified)',[2.0052938,1.1944509],-2.0239884,1)

In [82]:
class Particle:
    def __init__(self):
        self.velocity = np.random.uniform(-1,1, size = Obj.dim)
        self.position = np.random.uniform(-5,5, size = Obj.dim)
        self.pos_best = None
        self.best_obj = np.inf
        self.obj = None

    def evaluate(self): #evaluate the particle using the objective function
        self.obj = Obj.func(self.position)

        if self.obj < self.best_obj: #check to see if the current position is an individual best
            self.pos_best = self.position
            self.best_obj = self.obj

    # update new particle velocity
    def update_velocity(self, pos_best_n, iter, max_iter, start_w = 0.9, end_w = 0.4): #pos_best_n is best pos of neighbour
        w = np.linspace(start_w, end_w, max_iter, endpoint = True)
        #weighting how much the particles should care for its own best solution vs its neighbors best solution
        c1 = 1        # cognative constant
        c2 = 1       # social constant

        r1 = np.random.uniform(0,2)
        r2 = np.random.uniform(0,2)
        vel_cognitive = c1*r1*(self.pos_best - self.position)
        vel_social = c2*r2*(pos_best_n - self.position)

        self.velocity = w[iter]*self.velocity + vel_cognitive + vel_social
        if np.linalg.norm(self.velocity)>2: #if the velocity is to big it will be shortend
          self.velocity = 2*self.velocity/np.linalg.norm(self.velocity)

    # update the particle position based off new velocity updates
    def update_position(self, bounds=Obj.bounds):
      self.position = self.position + self.velocity
      
      if bounds == None:
        return self.position
      else:
        for i in range(Obj.dim):
            self.position[i] = max(min(self.position[0],bounds[i][1]),bounds[i][0])
        return self.position

In [83]:
class Swarm():
    def __init__(self,num_particles,maxiter, top):

        #data to save during the simulation
        self.top = top
        self.best_p  = np.inf          
        self.pos_best = []

        self.sim_pos = []  
        self.sim_vel = []  

        swarm=[]
        for i in range(num_particles): #initiate the swarm
            swarm.append(Particle())

        for i in range(maxiter): #evaluate all the particles
          for j in range(num_particles):
            swarm[j].evaluate()
            if swarm[j].obj  < self.best_p:
                self.best_p = swarm[j].obj 
                self.pos_best.append(swarm[j].position)
            else:
              self.pos_best.append(self.pos_best[-1])
          
          pos = []
          obj = []
          vel = []
          for j in range(num_particles):
            pos.append(swarm[j].position)
            obj.append(swarm[j].best_obj)
            vel.append(swarm[j].velocity)

          self.sim_pos.append(pos)          
          self.sim_vel.append(vel) 

          T = cKDTree(pos) #creating a tree for all the postions so to make it more easy to find a particles neighbours

          for j in range(num_particles):
            temp_value = np.inf
            temp_pos = None
            index = T.query(swarm[j].position,k=3)[-1]
            for k in index:#checking witch neighbour (including itself) has the best solution in its memory
              if obj[k] < temp_value:
                temp_pos = pos[k]
            
            if self.top == 'Ring':#using the closest 2 particles
              swarm[j].update_velocity(temp_pos,i,maxiter)
            if self.top == 'Global':#using all the particles as neighbours
              swarm[j].update_velocity(self.pos_best[-1],i,maxiter)
            swarm[j].update_position()

        # print final results
        print('Found position: ', self.pos_best[-1])
        print('Found objective: ', self.best_p)

In [84]:
S = Swarm(10,200,'Global')
print('Best known position: ',Obj.anal_sol)
print('Best known objective: ',Obj.anal_obj)

Found position:  [2.07834782 1.27942569]
Found objective:  -2.3758997791674754
Best known position:  [2.0052938, 1.1944509]
Best known objective:  -2.0239884


In [85]:
for j in range(len(S.sim_pos)): #saving each iteration as a figure
  r = 3
  xlist = np.linspace(-r , r , 500)
  ylist = np.linspace(-r , r , 500)

  X, Y = np.meshgrid(xlist, ylist)

  Z = Obj.func([X,Y])

  fig,ax=plt.subplots(1,1)
  cp = ax.contourf(X, Y, Z,levels = np.arange(-3,4,0.5))
  fig.colorbar(cp) 

  x_coord = []
  y_coord = []

  for i in range(len(S.sim_pos[j])):
    ax.plot(S.sim_pos[j][i][0],S.sim_pos[j][i][1],'ro', color = 'k')
    ax.arrow(S.sim_pos[j][i][0],S.sim_pos[j][i][1],S.sim_vel[j][i][0],S.sim_vel[j][i][1], color = 'k', length_includes_head = True)
    x_coord.append(S.sim_pos[j][i][0])
    y_coord.append(S.sim_pos[j][i][1])
    ax.set_title('Topology: '+str(S.top)+'\n'+'Objective Space Iteration: '+str(j)+'/'+str(len(S.sim_pos)))

  ax.plot(Obj.anal_sol[0], Obj.anal_sol[1],'ro', color ='r', label ='Best solution')
  ax.plot(S.pos_best[j][0], S.pos_best[j][1],'ro', color ='b', label ='Found solution')

  rim = 0.2

  ax.set_xlim(min(min(xlist),min(x_coord))-rim,max(max(xlist),max(x_coord))+rim)
  ax.set_ylim(min(min(ylist),min(y_coord))-rim,max(max(ylist),max(y_coord))+rim)

  ax.set_xlabel('x')
  ax.set_ylabel('y')
  ax.legend()
  fig.savefig('{}.png'.format(j+1))
  plt.close()

In [86]:
png_list = ['{}.png'.format(i+1) for i in range(len(S.sim_pos))] #making a gif of all the figures
with imageio.get_writer('mygif.gif', mode='I',duration=0.25) as writer:
    for filename in png_list:
        image = imageio.imread(filename)
        writer.append_data(image)